<a href="https://www.kaggle.com/code/huudatlego/train-model-cicids2017-cleaned?scriptVersionId=269608983" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Cell 1: Import Thư viện và Load Dữ liệu**

In [ ]:
# --- 1. IMPORT CÁC THƯ VIỆN CẦN THIẾT ---
import pandas as pd
import numpy as np
import joblib # Dùng để lưu mô hình

# Thư viện của Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

print("Các thư viện đã được import thành công!")

# --- 2. LOAD DỮ LIỆU SẠCH ---
# Thay thế '/kaggle/input/cicids2017-cleaned-data/cicids2017_cleaned.csv' 
# bằng đường dẫn chính xác đến file của bạn sau khi add data.
try:
    file_path = '/kaggle/input/cicids2017-cleaned-data/cicids2017_cleaned.csv'
    df = pd.read_csv(file_path)
    print("Đã tải thành công file dữ liệu sạch!")
    print(f"Bộ dữ liệu có {df.shape[0]} dòng và {df.shape[1]} cột.")
except FileNotFoundError:
    print("LỖI: Không tìm thấy file dữ liệu. Vui lòng kiểm tra lại đường dẫn file trong phần 'Add Input'.")

# Hiển thị 5 dòng đầu tiên để kiểm tra
df.head()

**Cell 2: Chuẩn bị Dữ liệu cho Huấn luyện**

In [ ]:
# --- 3. TÁCH BIẾN ĐỘC LẬP (X) VÀ BIẾN PHỤ THUỘC (y) ---
# X là tất cả các cột trừ cột 'Attack Type'
X = df.drop('Attack Type', axis=1)

# y là cột 'Attack Type'
y = df['Attack Type']

print("Đã tách X và y.")
print("Số lượng đặc trưng (features) trong X:", X.shape[1])

# --- 4. MÃ HÓA NHÃN (LABEL ENCODING) ---
# Chuyển các nhãn dạng chữ (ví dụ: 'Normal Traffic') thành số (ví dụ: 0, 1, 2...)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# In ra để xem các lớp và mã số tương ứng
print("\nCác lớp và mã số tương ứng:")
for index, label in enumerate(le.classes_):
    print(f"{label}: {index}")
    
# --- 5. CHIA DỮ LIỆU TRAIN/TEST (Tỷ lệ 70/30) ---
# Chia dữ liệu để có tập huấn luyện và tập kiểm thử riêng biệt
# stratify=y_encoded đảm bảo tỷ lệ các loại tấn công trong tập train và test là như nhau
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y_encoded, 
    test_size=0.3, 
    random_state=42, 
    stratify=y_encoded
)

print(f"\nKích thước tập huấn luyện (train): {X_train.shape[0]} dòng")
print(f"Kích thước tập kiểm thử (test): {X_test.shape[0]} dòng")

# --- 6. SCALING DỮ LIỆU BẰNG STANDARDSCALER ---
# Chuẩn hóa dữ liệu để các đặc trưng có cùng thang đo, giúp mô hình học tốt hơn
scaler = StandardScaler()

# Dùng fit_transform trên tập train để học các tham số scaling
X_train_scaled = scaler.fit_transform(X_train)

# Chỉ dùng transform trên tập test để áp dụng các tham số đã học từ tập train
X_test_scaled = scaler.transform(X_test)

print("\nĐã thực hiện scaling dữ liệu thành công!")

**Cell 3: Huấn luyện và Đánh giá Mô hình Random Forest**

In [ ]:
# --- 7. XÂY DỰNG VÀ HUẤN LUYỆN MÔ HÌNH RANDOM FOREST ---
# Khởi tạo mô hình với các tham số cơ bản
# n_jobs=-1 sẽ sử dụng tất cả các nhân CPU có sẵn để tăng tốc độ huấn luyện
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, verbose=1)


### THÊM MỚI: THỰC HIỆN KIỂM ĐỊNH CHÉO (CROSS-VALIDATION) ###
from sklearn.model_selection import cross_val_score
print("Bắt đầu thực hiện Kiểm định chéo 5-fold trên tập huấn luyện...")
# cv=5 nghĩa là chia tập train ra 5 phần
# n_jobs=-1 để chạy song song, tăng tốc
cv_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=5, n_jobs=-1)
print("...Kiểm định chéo hoàn tất!")
print(f"--> Cross-Validation Scores: {cv_scores}")
print(f"--> Cross-Validation Mean Score: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}\n")


print("Bắt đầu quá trình huấn luyện mô hình Random Forest trên toàn bộ tập train...")
# Huấn luyện mô hình trên dữ liệu đã được scaling
rf_model.fit(X_train_scaled, y_train)
print("...Huấn luyện hoàn tất!")


# --- 8. ĐÁNH GIÁ HIỆU NĂNG MÔ HÌNH ---
print("\nĐang thực hiện dự đoán trên tập kiểm thử...")
# Dùng mô hình đã huấn luyện để dự đoán trên tập test
y_pred = rf_model.predict(X_test_scaled)
print("...Dự đoán hoàn tất!")


### CẬP NHẬT: In báo cáo với 4 chữ số thập phân ###
print("\nBáo cáo Phân loại (Classification Report) với 4 chữ số thập phân:")
# Thêm tham số digits=4
print(classification_report(y_test, y_pred, target_names=le.classes_, digits=4))


# In ma trận nhầm lẫn (Confusion Matrix)
print("\nMa trận Nhầm lẫn (Confusion Matrix):")
cm = confusion_matrix(y_test, y_pred)
# Vẽ heatmap cho ma trận nhầm lẫn để dễ nhìn hơn
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Dự đoán (Predicted)')
plt.ylabel('Thực tế (Actual)')
plt.title('Confusion Matrix Heatmap')
plt.show()

# In độ chính xác tổng thể (tính lại từ cm để chắc chắn)
accuracy_from_cm = np.trace(cm) / np.sum(cm)
print(f"\nĐộ chính xác tổng thể (tính từ Ma trận Nhầm lẫn): {accuracy_from_cm * 100:.4f}%")

**Cell 4: Lưu lại Mô hình và các Công cụ**

In [ ]:
# --- 9. LƯU LẠI CÁC "SẢN PHẨM" ĐỂ TRIỂN KHAI ---
# Tạo tên file
model_filename = 'botnet_model.pkl'
scaler_filename = 'scaler.pkl'
encoder_filename = 'label_encoder.pkl'

# Dùng joblib để lưu các đối tượng Python
joblib.dump(rf_model, model_filename)
joblib.dump(scaler, scaler_filename)
joblib.dump(le, encoder_filename)

print(f"Đã lưu thành công 3 files vào thư mục /kaggle/working/:")
print(f"- Mô hình: {model_filename}")
print(f"- Scaler: {scaler_filename}")
print(f"- Label Encoder: {encoder_filename}")